## Categorical Feature Encoding Project - 머신러닝 편 (성능개선 ver.2)
- 앞의 베이스라인 모델을 만들며 범주형 변수의 인코딩(Encoding)을 다루었다.
- 이번 노트북에서는 성능 향상을 위해 이전의 베이스라인 모델에 몇 가지 경우를 더 추가해보았다.
    - 1) 명목형 변수(`nom_*` 의 변수 개수 조정) : **nom_9 변수 제거**
    - 2) 로지스틱 회귀의 하이퍼파라미터 튜닝

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
train = pd.read_csv('train.csv', index_col = 'id')
test = pd.read_csv('test.csv', index_col = 'id')

display(train.head())
display(test.head())

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,T,Y,Green,Triangle,Snake,Finland,Bassoon,...,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2,0
1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,Piano,...,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8,0
2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,Theremin,...,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2,0
3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,Oboe,...,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1,1
4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,Oboe,...,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8,0


,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
id,,,,,,,,,,,,,,,,,,,,,
300000,0,0,1,T,Y,Blue,Triangle,Axolotl,Finland,Piano,...,9d117320c,3c49b42b8,2,Novice,Warm,j,P,be,5,11
300001,0,0,0,T,N,Red,Square,Lion,Canada,Piano,...,46ae3059c,285771075,1,Master,Lava Hot,l,A,RP,7,5
300002,1,0,1,F,Y,Blue,Square,Dog,China,Piano,...,b759e21f0,6f323c53f,2,Expert,Freezing,a,G,tP,1,12
300003,0,0,1,T,Y,Red,Star,Cat,China,Piano,...,0b6ec68ff,b5de3dcc4,1,Contributor,Lava Hot,b,Q,ke,2,3
300004,0,1,1,F,N,Red,Trapezoid,Dog,China,Piano,...,f91f3b1ee,967cfa9c9,3,Grandmaster,Lava Hot,l,W,qK,4,11


In [3]:
# 인코딩은 train과 test에 모두 해줄 것이므로 먼저 두개를 이어붙인다. target은 따로 빼서 저장
all_data = pd.concat([train, test], ignore_index = True)
all_data.drop(columns = 'target', axis = 1, inplace = True)
all_data.head(3)

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
0,0,0,0,T,Y,Green,Triangle,Snake,Finland,Bassoon,...,c389000ab,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2
1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,Piano,...,4cd920251,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8
2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,Theremin,...,de9c9f684,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2


In [4]:
print(all_data.shape)

(500000, 23)


In [5]:
# target
train_target = train['target']

## 2nd. 로지스틱 회귀 모델의 `하이퍼 파라미터 튜닝`
- nom_9는 제거
- 전체 코드는 이전에 쓴것과 똑같으나, 로지스틱 회귀 모델의 하이퍼 파라미터 튜닝 부분만 바꿨다.
    - 범주형 데이터 전처리 부분은 이전 ver.1 노트북에서 만든 전처리 함수 그대로 사용
    - 하이퍼파라미터 튜닝용 함수 만들기

In [6]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from scipy import sparse

### 범주형 데이터 전처리 함수
def cat_preprocessing(df):
    # 반환되는 값은 train_sprs, test_sprs
    
    # bin_* 변수 전처리
    bin_cols = [f'bin_{i}' for i in range(0, 5)]
    bin_data = df[bin_cols]
    bin_data['bin_3'] = bin_data['bin_3'].apply(lambda x: 1 if x == 'T' else 0)
    bin_data['bin_4'] = bin_data['bin_4'].apply(lambda x: 1 if x == 'Y' else 0)
    
    # ord_* 변수 전처리
    ord_cols = [f'ord_{i}' for i in range(0, 6)] 
    ord_data = df[ord_cols]
    ord1_enc = {'Novice': 0, 'Contributor': 1, 'Expert': 2, 'Master': 3, 'Grandmaster': 4}
    ord2_enc = {'Freezing': 0 , 'Cold': 1, 'Warm': 2,  'Hot': 3, 'Boiling Hot': 4, 'Lava Hot': 5}
    ord_data.loc[:, 'ord_1'] = ord_data.loc[:, 'ord_1'].map(ord1_enc)
    ord_data.loc[:, 'ord_2'] = ord_data.loc[:, 'ord_2'].map(ord2_enc)
        ## 숫자 순위 부여
    encoder = LabelEncoder()
    ord_data['ord_3'] = encoder.fit_transform(ord_data['ord_3'].values)
    ord_data['ord_4'] = encoder.fit_transform(ord_data['ord_4'].values)
    ord_data['ord_5'] = encoder.fit_transform(ord_data['ord_5'].values)
        ## 숫자 스케일링
    scaler = StandardScaler()
    ord_data_scaled = scaler.fit_transform(ord_data)
    
    # 앞에서 만든 bin_data & ord_data 합침
    df2 = pd.concat([bin_data, pd.DataFrame(ord_data_scaled, columns = ord_data.columns)], axis = 1)
    
    nom_cols = [f'nom_{i}' for i in range(0, 9)] # nom_9 변수 제거 
    nm_list = ['day', 'month']
    nom_cols.extend(nm_list)
    nom_data = df[nom_cols]
   
    encoder = OneHotEncoder()
    nom_data_enc = encoder.fit_transform(nom_data)
    
    # df2와 nom_data_enc를 결합
    all_sprs = sparse.hstack([sparse.csr_matrix(df2), nom_data_enc], format = 'csr')
    
    # all_sprs는 앞 부분은 train / 뒷 부분은 test 데이터이므로 분할
    train_sprs = all_sprs[:len(train), :]
    test_sprs = all_sprs[len(train):, :]
    
    return train_sprs, test_sprs

In [7]:
## 적용 
train_sprs, test_sprs = cat_preprocessing(all_data)

### 하이퍼 파라미터를 바꿔보며, 성능을 바꿔보자!
- 하이퍼파라미터: `penalty`, `C`, `max_iter`

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

## 하이퍼파라미터 튜닝용 함수
def logistic_tuning(train_sprs, y, params):
    
    model = LogisticRegression(random_state = 99)
    # 파라미터 튜닝(train data 전체를 넣어서 5-fold cv)
    grid = GridSearchCV(model, params, scoring = 'roc_auc', cv = 5)
    grid.fit(train_sprs, y)
    
    print(grid.best_params_)
    print(grid.best_score_)
    
    return grid.best_estimator_

In [9]:
%%time
param1 = {'penalty':['l2', 'l1'],  'C':[0.01, 0.1, 1, 5, 10], 'max_iter': [100, 500]}
logistic_tuning(train_sprs, train_target, params = param1)

{'C': 0.1, 'max_iter': 500, 'penalty': 'l2'}
0.8011685328386255
Wall time: 12min 43s


LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=99, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

- 결과를 보면 `penalt = 'l2'` 일 때(디폴트), `C = 0.1`, `max_iter = 500`일 때가 가장 성능이 좋았다.
- penalty = l2는 디폴트이므로 이번엔 C와 max_iter 값만 세분화해서 다시 튜닝해보자!

In [10]:
%%time

# 좀더 세분화하여 파라미터 튜닝 (5-fold cv)
param2 = {'C':[0.05, 0.1, 0.125, 0.15, 0.5], 'max_iter': [300, 500, 700, 900]}
logistic_tuning(train_sprs, train_target, params = param2)

{'C': 0.125, 'max_iter': 300}
0.8011892078562894
Wall time: 22min 34s


LogisticRegression(C=0.125, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=300,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=99, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [11]:
%%time

# 좀더 세분화하여 파라미터 튜닝 (5-fold cv)
param3 = {'C':[0.1, 0.125, 0.15], 'max_iter': [200, 300, 400, 500]}
logistic_tuning(train_sprs, train_target, params = param3)

{'C': 0.125, 'max_iter': 300}
0.8011892078562894
Wall time: 12min 13s


LogisticRegression(C=0.125, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=300,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=99, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

- 결과적으로 `C = 0.125`, `max_iter = 300`일 때 train AUC 값이 **0.801189**로 가장 좋았다. 
    - 그리고 사실 결과를 보면 알겠지만, 하이퍼 파라미터로 튜닝해도 시간만 10~20분 걸릴 뿐 성능은 크게 달라짐이 없다.
    - train AUC 값은 5-fold cv를 했을 때 0.801이라 정확한 train AUC는 아니긴 하지만, 그리드서치를 하기 전이 0.804였음을 감안하면 성능이 소폭 감소했다.

### 모델 튜닝 후 제출 결과

In [12]:
# 파라미터 튜닝 결과 가장 AUC 값이 좋았던 건 이렇게 뽑으면 된다! 
model = logistic_tuning(train_sprs, train_target, params = param3)

{'C': 0.125, 'max_iter': 300}
0.8011892078562894


In [13]:
y_pred = model.predict_proba(test_sprs)[:, 1]

# sub 데이터에 담아서 제출
sub = pd.read_csv('sample_submission.csv')
del sub['target']
sub['target'] = y_pred
sub

,id,target
0,300000,0.379854
1,300001,0.697356
2,300002,0.137984
3,300003,0.491129
4,300004,0.860734
...,...,...
199995,499995,0.260177
199996,499996,0.135663
199997,499997,0.310009
199998,499998,0.629618


In [14]:
sub.to_csv('sub_sy_final.csv', index = False)

### 결과: test data에 대한 Score
- 하이퍼파라미터 튜닝 결과, **public: 0.80539, private: 0.79917**
    - k-fold를 했을 때 나온 train AUC(0.801)보다 더 성능이 좋았다.
    - ver.1에서 만들었던 결과인 0.804보다 소폭 상승한 스코어이다.
- 로지스틱 회귀 모델을 사용할 경우엔 `L2 규제, C = 0.125, max_iter = 300` 을 쓰는 게 가장 좋았다!